In [28]:
import requests
import time
from datetime import date,datetime,timezone
import pandas as pd
import plotly.graph_objects as go

# settings
finn_key="btv5stv48v6q7nvmtsjg"
base_url = "https://finnhub.io/api/v1/stock/candle"

In [75]:
# code to pull daily stock prices for 'symbol'
# over the given date range
def bollinger(symbol,start_date,end_date):
    # format the API call with passed in parameters
    params = {'symbol':symbol,
              'resolution':'D',
              'from':start_date,
              'to':end_date,
              'token':finn_key}
    response = requests.get(base_url,params=params).json()
    # convert to dataframe
    df = pd.DataFrame(response)
    # make a date column that is human readable
    df['date'] = [datetime.fromtimestamp(d) for d in df['t']]
    # typical price is the average of the high, low and close
    df['tp'] = (df.h + df.l + df.c)/3.
    # 20 day rolling average of the typical price
    df['ma20']  = df.tp.rolling(20).mean()
    # 20 day rolling standard dev of typical price
    df['std20'] = df.tp.rolling(20).std()
    # define upper and lower Bollinger bands as 
    # 2 deviations above and below the moving average
    df['upper'] = df['ma20'] + 2*df['std20']
    df['lower'] = df['ma20'] - 2*df['std20']
    # create a plot using plotly with upper and lower Bollinger bands,
    # a plot for the moving average and a candlestick plot 
    # for the stock we're interested in
    fig = go.Figure(data = [go.Scatter(x=df['date'],y=df['upper'],line_color='gray',name='Bollinger uppper'),
                        go.Scatter(x=df['date'],y=df['lower'],line_color='gray',name='Bollinger lower',fill='tonexty'),
                        go.Scatter(x=df['date'],y=df['ma20'],line=dict(color='gray',dash='dash'),name='20d moving avg'),
                        go.Candlestick(x=df['date'],
                                       open=df['o'],
                                       high=df['h'],
                                       low=df['l'],
                                       close=df['c'],
                                       name=symbol)])
    # allow for adjusting the y-axis
    fig.update_yaxes(fixedrange = False)
    return fig

In [76]:
stocks = ['AMZN','AAPL','MSFT','TSLA']
#Oct 9, 2020
end_date = 1602277200 
#Oct 9, 2018
start_date = end_date-2*365*24*3600 
for stock in stocks:
    fig = bollinger(stock,start_date,end_date)
    fig.show()
            